<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLDM-2021/blob/master/09-convolutions-and-regularization/MLDM_2021_seminar09_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import skimage
import os
import matplotlib.pyplot as plt
import imageio
import numpy as np
import skimage.io
import skimage.transform
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from sklearn.metrics import classification_report
from scipy.ndimage.filters import convolve
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In this notebook we are going to explore a dataset with [annotated images of bees](https://www.kaggle.com/jenny18/honey-bee-annotated-images) from various locations of US, captured over several months during 2018, at different hours, from various bees subspecies, and with different health problems.

> The original batch of images was extracted from still time-lapse videos of bees. By averaging the frames to calculate a background image, each frame of the video was subtracted against that background to bring out the bees in the forefront. The bees were then cropped out of the frame so that each image has only one bee. Because each video is accompanied by a form with information about the bees and hive, the labeling process is semi-automated. Each video results in differing image crop quality levels. This dataset will be updated as more videos and data become available.

# EDA

Let's start from an Exploratory Data Analysis (EDA) where we highlight labels' distributions. We will try to perform subspecies classification through the notebook, but you may try use "health" as a target as well. We are going to work with images only, however you may try to use other features as well, so you may find a way more [detailed EDA in the notebook](https://www.kaggle.com/gpreda/honey-bee-subspecies-classification/notebook), that is used as a baseline for this seminar.

The data contains the following values:

* file - the image file name;
* date - the date when the picture was taken;
* time - the time when the picture was taken;
* location - the US location, with city, state and country names;
* zip code - the ZIP code associated with the location;
* subspecies - the subspecies to whom the bee in the current image belongs;
* health - this is the health state of the bee in the current image;
* pollen_carrying - indicates if the picture shows the bee with pollen attached to the legs;
* caste - the bee caste

In [1]:
#!wget https://raw.githubusercontent.com/HSE-LAMBDA/MLDM-2021/main/09-convolutions-and-regularization/bees_data.zip

In [ ]:
#!unzip bees_data.zip

In [ ]:
bees=pd.read_csv('bee_data.csv', index_col=False, dtype={'subspecies':'category', 'health':'category','caste':'category'})

In [ ]:
bees.head()

In [ ]:
# Check whether image for some particular given discription exists
img_exists = bees['file'].apply(lambda f: os.path.exists('bee_imgs/bee_imgs/' + f))
bees = bees[img_exists]

Let's check the distributions of bees by categories that we are going to use as our target labels and have a look at the images. (1 point)

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,10))

#Plot a bar or pie chart of Subspecies labels
<YOUR CODE>
ax[0].set_title('Subspecies')

#Same plot fot the health label
<YOUR CODE>
ax[1].set_title('Health')

plt.show()

In [ ]:
img_folder = 'bee_imgs/bee_imgs/'

f_figsize = (14,3)

subspecies = bees['subspecies'].cat.categories
f, ax = plt.subplots(nrows=1,ncols=subspecies.size - 2, figsize=f_figsize)

# Draw the first found bee of given subpecies
i=0
for s in subspecies[2:]:
    if s == 'healthy': continue
    file=img_folder + bees[bees['subspecies']==s].iloc[0]['file']
    im=imageio.imread(file)
    ax[i].imshow(im, resample=True)
    ax[i].set_title(s, fontsize=8)
    i+=1
    
plt.suptitle("Subspecies of Bee")
plt.tight_layout()
plt.show()

# Sample some healthy objects to have a look at
ncols = 5
healthy = bees[bees['health'] == 'healthy'].sample(ncols)
f, ax = plt.subplots(nrows=1,ncols=ncols, figsize=f_figsize)

for i in range(0, ncols): 
    file = img_folder + healthy.iloc[i]['file']
    ax[i].imshow(imageio.imread(file))

plt.suptitle("Healthy Bees")
plt.tight_layout()
plt.show()

health_cats = bees['health'].cat.categories
f, ax = plt.subplots(1, health_cats.size-1, figsize=f_figsize)

# Draw the first found bee with a particulat health issue
i=0
for c in health_cats:
    if c == 'healthy': continue
    bee = bees[bees['health'] == c].sample(1).iloc[0]
    ax[i].imshow(imageio.imread(img_folder + bee['file']))
    ax[i].set_title(bee['health'], fontsize=8)
    i += 1
    
plt.suptitle("Sick Bees")    
plt.tight_layout()
plt.show()

# Imbalanced labels

Accorgin to the plots above labels are imbalanced, so let's fix it on our own. Splitting should be done before balancing to avoid putting the same upsampled Bee to both train and test.

In [ ]:
def split_balance(bees, target):
    # Split to train and test before balancing
    train_bees, test_bees = train_test_split(bees, random_state=24)

    # Split train to train and validation datasets
    train_bees, val_bees = train_test_split(train_bees, test_size=0.1, random_state=24)

    #Balance by subspecies to train_bees_bal_ss dataset
    # Number of samples in each category
    ncat_bal = int(len(train_bees)/train_bees[target].cat.categories.size)
    train_bees_bal = train_bees.groupby(target, as_index=False).apply(lambda g:  g.sample(ncat_bal, replace=True)).reset_index(drop=True)
    return(train_bees_bal, val_bees, test_bees)

In [ ]:
train_bees_bal, val_bees, test_bees = split_balance(bees, 'subspecies')

In [ ]:
f, axs = plt.subplots(2,1, figsize=(12,8), sharex = True)

# Oridigan data
ax = bees['subspecies'].value_counts().plot(kind='bar', rot = 0, ax=axs[0])
ax.set_title('Original dataset')
ax.set_ylabel('Count')

# Balanced train
ax = train_bees_bal['subspecies'].value_counts().plot(kind='bar', rot = 0, ax=axs[1])
ax.set_title('Train dataset after balancing')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

# Subspecies classification

In [ ]:
# Some default network parameters
IMAGE_WIDTH, IMAGE_HEIGHT = 100, 100
KERNEL_SIZE = 3
IMAGE_CHANNELS = 3
RANDOM_STATE = 1337
N_EPOCH = 5
BATCH_SIZE = 64
MAX_POOL_DIM = 2

Here you can find a few auxiliary functions that we will help us through the model-building procedure. The dataset contains images of different shapes. The function below helps us to read images from the image-files and scale all images to IMAGE_WIDTH x IMAGE_HEIGHT x IMAGE_CHANNELS

In [ ]:
def read_img(file, img_folder='bee_imgs/bee_imgs/'):    
    img = skimage.io.imread(img_folder + file)
    img = skimage.transform.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT), mode='reflect')
    return img[:,:,:IMAGE_CHANNELS]

We are going to evaluate the model, estimating the training error and accuracy and also the validation error and accuracy. It may help us to decide how our workflow is going, e.g. if we have a high bias, we may firstly try to improve our model so that it will learn better the train images dataset. Otherwise, in case we have a small bias but large variance, that means our model faces overfitting. Based on these kind of observation, we make decission for how to adjust the model.

`tf.keras.preprocessing.image.ImageDataGenerator` may help us to generate batches of tensor-image-data with [real-time data augmentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator).

In [ ]:
def prepare2train(train_bees, val_bees, test_bees, target):
    # Bees already splitted to train, validation and test
    # Load and transform images to have equal width/height/channels. 
    # read_img function is defined to use in both health and subspecies. 
    # Use np.stack to get NumPy array for CNN input

    # Train data
    train_X = np.stack(train_bees['file'].apply(read_img))
    train_y  = pd.get_dummies(train_bees[target], drop_first=False)

    # Validation during training data to calculate val_loss metric
    val_X = np.stack(val_bees['file'].apply(read_img))
    val_y = pd.get_dummies(val_bees[target], drop_first=False)

    # Test data
    test_X = np.stack(test_bees['file'].apply(read_img))
    test_y = pd.get_dummies(test_bees[target], drop_first=False)

    # Data augmentation - a little bit rotate, zoom and shift input images.
    generator = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            rotation_range=180,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)
    generator.fit(train_X)
    return (generator, train_X, val_X, test_X, train_y, val_y, test_y)

In [ ]:
generator, train_X, val_X, test_X, train_y, val_y, test_y = prepare2train(train_bees_bal, val_bees, test_bees, 'subspecies')

In [ ]:
# We'll stop training if no improvement after some epochs
earlystopper1 = keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)

# Save the best model during the traning
checkpointer1 = keras.callbacks.ModelCheckpoint('best_model1.h5',
                                                monitor='val_accuracy',
                                                verbose=1,
                                                save_best_only=True,
                                                save_weights_only=True)
# Build CNN model
model1=Sequential()
model1.add(Conv2D(16, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS), activation='relu', padding='same'))
model1.add(MaxPool2D(MAX_POOL_DIM))
model1.add(Conv2D(16, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model1.add(Flatten())
model1.add(Dense(train_y.columns.size, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

We train the first model using `Model.fit_generator`, however it is possible to call `Model.fit`, which supports generators, in case of new release of TF, and a predefined batch size. Basically, `steps_per_epoch` may be increased as we use augmentation.

In [ ]:
training1 = model1.fit_generator(generator.flow(train_X,train_y, batch_size=BATCH_SIZE),
                                 epochs=N_EPOCH,
                                 validation_data=[val_X, val_y],
                                 steps_per_epoch=50,
                                 callbacks=[earlystopper1, checkpointer1])
# Get the best saved weights
model1.load_weights('best_model1.h5')

In [ ]:
def eval_model(training, model, test_X, test_y, target):
    
    ## Trained model analysis and evaluation
    f, ax = plt.subplots(2,1, figsize=(5,5))
    ax[0].plot(training.history['loss'], label="Loss")
    ax[0].plot(training.history['val_loss'], label="Validation loss")
    ax[0].set_title('%s: loss' % target)
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].legend()
    
    # Accuracy
    ax[1].plot(training1.history['accuracy'], label="Accuracy")
    ax[1].plot(training1.history['val_accuracy'], label="Validation accuracy")
    ax[1].set_title('%s: accuracy' % target)
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy')
    ax[1].legend()
    plt.tight_layout()
    plt.show()

    # Accuracy by subspecies
    test_pred = model.predict(test_X)    
    acc_by_subspecies = np.logical_and((test_pred > 0.5), test_y).sum()/test_y.sum()
    acc_by_subspecies.plot(kind='bar', title='Accuracy by %s' % target)
    plt.ylabel('Accuracy')
    plt.show()

    # Print metrics
    print("Classification report")
    test_pred = np.argmax(test_pred, axis=1)
    test_truth = np.argmax(test_y.values, axis=1)
    print(classification_report(test_truth, test_pred, target_names=test_y.columns))

    # Loss function and accuracy
    test_res = model.evaluate(test_X, test_y.values, verbose=0)
    print('Loss function: %s, accuracy:' % test_res[0], test_res[1])

In [ ]:
eval_model(training1, model1, test_X, test_y, 'subspecies')

# Kernel visualization

Function below displays how input sample image looks like after you apply convolution using some particular kernel. 


In [ ]:
def visualize_layer_kernels(img, conv_layer, title):
    # Extract kernels from given layer
    weights1 = conv_layer.get_weights()
    kernels = weights1[0]
    kernels_num = kernels.shape[3]
    
    # Each row contains 3 images: kernel, input image, output image
    f, ax = plt.subplots(kernels_num, 3, figsize=(10, kernels_num*4))

    for i in range(0, kernels_num):
        # Get kernel from the layer and draw it
        kernel=kernels[:,:,:3,i]
        ax[i][0].imshow((kernel * 255).astype(np.uint8), vmin=0, vmax=255)
        ax[i][0].set_title("Kernel %d" % i, fontsize = 9)
        
        # Get and draw sample image from test data
        ax[i][1].imshow((img * 255).astype(np.uint8), vmin=0, vmax=255)
        ax[i][1].set_title("Before", fontsize=8)
        
        # Filtered image - apply convolution
        img_filt = convolve(img, kernel)
        ax[i][2].imshow((img_filt * 255).astype(np.uint8), vmin=0, vmax=255)
        ax[i][2].set_title("After", fontsize=8)
        
    plt.suptitle(title)
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()

In [ ]:
# Sample image to visualize convolution
idx = np.random.randint(0,len(test_X)-1)
img = test_X[idx,:,:,:]
# Take 1st convolutional layer and look at it's filters
conv1 = model1.layers[0]
img = visualize_layer_kernels(img, conv1, "Subspecies CNN. Layer 0")

In [ ]:
# Sample another image to visualize convolutoin
idx = np.random.randint(0,len(test_y)-1)
img = test_X[idx,:,:,:]
# Take another convolutional layer and look at it's filters
conv2 = model1.layers[2]
res = visualize_layer_kernels(img, conv2, "Subspecies CNN. Layer 2")

# Dropout

Adding additional data will only slightly increase the accuracy of the training set.
To reduce the loss of the validation set (which is a sign of overfitting), we can have such strategies as:

* add Dropout layers;
* introduce strides;
* modify the Hyperparameters 

Let's pick up the first one. 
The Dropout layer randomly sets input units to 0 with a frequency of `rate` at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - `rate`) such that the sum over all inputs is unchanged.

Note that the Dropout layer only applies when training is set to True such that no values are dropped during inference. When using model.fit, training will be appropriately set to `True` automatically, and in other contexts, you can set the kwarg explicitly to True when calling the layer.

In [ ]:
tf.random.set_seed(RANDOM_STATE)
layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
data = np.arange(10).reshape(5, 2).astype(np.float32)
print(data)

In [ ]:
outputs = layer(data, training=True)
print(outputs)

Now plug `Dropout` into the model. Should we add it as a last layer?

In [ ]:
model2=Sequential()
#Use the same architecture as it was in model1 case, but add Dropout layers. (1 point)
#You may increase the number of epoch to 10, but don't change other parameters and random state.
#(2 point for best-performing shared architecture)
#model2.add(...)
<YOUR CODE>
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.summary()


In [ ]:
training2 = model2.fit_generator(generator.flow(train_X,train_y, batch_size=BATCH_SIZE),
                                     epochs=N_EPOCH,
                                     validation_data=[val_X, val_y],
                                     steps_per_epoch=50,
                                     callbacks=[earlystopper1, checkpointer1])

In [ ]:
eval_model(training2, model2, test_X, test_y, 'subspecies')